In [1]:
# https://dacon.io/competitions/official/235930/codeshare/5508?page=1&dtype=recent


In [2]:
import random
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
epochs = 100
lr = 1e-2
batch_size = 256
momentum = 0.9
weight_decay = 1e-4
seed = 42
NUM_WORKERS = 8

param = {
            'epochs' : epochs,
            'lr' :lr,
            'batch_size' : batch_size,
            'momentum' : momentum,
            'weight_decay' : weight_decay
        }

In [6]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed)    

In [7]:
train_df = pd.read_csv('../dataset/train.csv')
val_df = pd.read_csv('../dataset/val.csv')
train_df = train_df.drop(columns=['ID'])
val_df = val_df.drop(columns=['ID'])
print(train_df.shape)

(113842, 30)


In [8]:
# class CDataset(Dataset) :
#     def __init__(self, df, eval_mode=False) :
#         self.df = df
#         self.eval_mode = eval_mode
#         if self.eval_mode :
#             self.labels = self.df['Class']
#             self.df = self.df.drop(columns=['Class'])
        
#     def __getitem__(self, idx) :
#         if self.eval_mode :
#             self.x = self.df.iloc[idx, 1:]
#             self.y = self.labels[idx]
#             return torch.tensor(self.x, dtype=torch.float32),torch.tensor(self.y)
        
#         else :
#             self.x = self.df.iloc[idx, 1:]
#             return torch.tensor(self.x, dtype=torch.float32)
        
#     def __len__(self) :
#         return len(self.df)
        

In [9]:
class CDataset(Dataset):
    def __init__(self, df, eval_mode=False):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [10]:
train_dataset = CDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=NUM_WORKERS)

val_dataset = CDataset(val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [11]:
x = next(iter(train_loader))
x.shape

torch.Size([256, 30])

In [12]:
x, y  = next(iter(val_loader))
x.shape

torch.Size([256, 30])

In [ ]:
y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
class AutoEncoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(30,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,30),
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)        
        
    def forward(self, x) :
        x = self.encoder(x)
        x = self.decoder(x)
        return x
        

In [15]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device, **param) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.batch_size = param['batch_size']
        self.epochs = param['epochs']
        self.lr = param['lr']
        
        self.criterion = nn.L1Loss().to(device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                _x = self.model(x)
                
                loss = self.criterion(x, _x)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                
            score = self.validation(self.model, 0.95)
            
            if self.scheduler is not None :
                self.scheduler.step(score)
            
            print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}]')
            for param_group in self.optimizer.param_groups:
                print(param_group['lr'])      
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}] lr [{self.scheduler.get_lr()}]')

            if best_score < score :
                best_score = score
                torch.save(self.model.state_dict(), '../saved/best_model.pth')
            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()
                
        return f1_score(true_y, pred_y, average='macro')

In [16]:
model = AutoEncoder()
model.eval()
optimizer = torch.optim.Adam(params=model.parameters(), lr = param['lr'])

# optimizer = torch.optim.SGD(model.parameters(), param['lr'],
#                             momentum=param['momentum'],
#                             weight_decay=param['weight_decay'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
# scheduler = StepLR(optimizer, step_size=50, gamma=0.2)


In [17]:
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device, **param)
# trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)

In [18]:
trainer.fit()

epoch :[0] train loss [0.18748281430662347] val score [0.5236718088870304]
0.01
epoch :[1] train loss [0.13025402413325363] val score [0.5426217169100375]
0.01
epoch :[2] train loss [0.1149567018231649] val score [0.5762491119793732]
0.01
epoch :[3] train loss [0.10682301049152117] val score [0.6752462221463755]
0.01
epoch :[4] train loss [0.10239242532949769] val score [0.6901265786034538]
0.01
epoch :[5] train loss [0.09763400109296434] val score [0.7246883762645999]
0.01
epoch :[6] train loss [0.09421416850907079] val score [0.7805557779616334]
0.01
epoch :[7] train loss [0.09167920908566271] val score [0.7903809848799157]
0.01
epoch :[8] train loss [0.08894564991586663] val score [0.8287186884323108]
0.01
epoch :[9] train loss [0.08733435996462789] val score [0.8422634702634115]
0.01
epoch :[10] train loss [0.08463396126969477] val score [0.8422634702634115]
0.01
epoch :[11] train loss [0.08181247819173203] val score [0.8519279892324237]
0.01
epoch :[12] train loss [0.0810327702191

Exception ignored in: <function _releaseLock at 0x7eff0a9a6e60>
Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt


KeyboardInterrupt: 